# Text Representation

In [2]:
import json
import pandas as pd
import gzip
import os
from urllib.request import urlopen

In [3]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-03-07 14:19:29--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu... 169.228.63.50
Connecting to deepyeti.ucsd.edu|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: 'meta_All_Beauty.json.gz.1'

meta_All_Beauty.jso 100%[===================>]   9.85M  2.78MB/s    in 4.3s    

2022-03-07 14:19:34 (2.31 MB/s) - 'meta_All_Beauty.json.gz.1' saved [10329961/10329961]



# Exercise 1

Load the [metadata file](https://nijianmo.github.io/amazon/index.html) and discard any item that was not rated by our subset of users (nor in training or test sets). Apply preprocessing (stemming and stopwords removal) to clean up the text from the "title". Report the vocabulary size before and after the preprocessing.

In [7]:
import os
import sys
sys.path.append('../')
import pickle
import pandas as pd

# Load TRAIN and TEST sets 
from Session_1 import training_data, test_data

# Load the METADATA (ITEMS)
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('meta_All_Beauty.json.gz')
print(len(df))

# Discard duplicates
df = df.drop_duplicates(subset=['asin']).reset_index(drop=True)
print(len(df))

# Discard items that weren't rated by our subset of users
item_in_training = df['asin'].isin(training_data.append(test_data)['asin'])
df = df[item_in_training]
print(len(df))

32892
32488
84


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

# <YOUR CODE HERE>

# Exercise 2

Representation in vector spaces.

## 2.1

Represent all the products from Exercise 1 in a TF-IDF space. Interpret the meaning of the TF-IDF matrix dimensions.

Tip: You may use the library [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

# <YOUR CODE HERE>

## 2.2

Compute and the cosine similarity between products with asin 'B000FI4S1E', 'B000LIBUBY' and 'B000W0C07Y'. Take a look at their features to see whether results make sense with their characteristics. 

# Exercise 3

Representation in vector spaces with contextual Word Embeddings.

## 3.1.

Represent all the products from Exercise 1 in a vector space using embeddings from a pre-trained BERT model. The final embedding of a product should be the average of the word embeddings from all the words in the 'title'. What is the vocabulary size of the model? What are the dimensions of the last hidden state?

Tip: you may install the transformers library and use their pretrained [BERT model uncased](https://huggingface.co/bert-base-uncased).

In [ ]:
# LOAD TRANSFORMER
"""
If you plan on using a pretrained model, it’s important to use the associated 
pretrained tokenizer: it will split the text you give it in tokens the same way
for the pretraining corpus, and it will use the same correspondence
token to index (that we usually call a vocab) as during pretraining.
"""

# % pip install transformers
import torch
import transformers
assert transformers.__version__ > '4.0.0'

from transformers import BertModel, BertTokenizerFast

# set-up environment
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")


modelname = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(modelname)
model = BertModel.from_pretrained(modelname).to(DEVICE)

# Print out the vocabulary size
# <YOUR CODE HERE>

In [ ]:
# REPRESENT PRODUCTS IN A VECTOR SPACE


def batch_encoding(sentences):
    # Since we're using padding, we need to provide the attention masks to our
    # model. Otherwise it doesn't know which tokens it should not attend to. 
    inputs = # <YOUR CODE HERE>
    # print(inputs) # Look at the padding and attention_mask

    outputs = model(**inputs)

    last_hidden_states = # <YOUR CODE HERE>

    return inputs, last_hidden_states
  
encoded_inputs, title_last_hidden_states = batch_encoding( # <YOUR CODE HERE> )

"""
Note that the control token [CLS] has been added 
at the beginning of each sentence, and [SEP] at the end. 
"""

# Now, let's mask out the padding tokens and compute the embedding vector of each product

# <YOUR CODE HERE>

## 3.2.

Compute and the cosine similarity between products with asin 'B000FI4S1E', 'B000LIBUBY' and 'B000W0C07Y'.